# Validation Set 1: diffuPy + PathMe  

In [1]:
import os
dir_path = os.path.dirname(os.path.realpath('__file__'))

In [2]:
from openpyxl import load_workbook
from collections import defaultdict
import pybel
import pybel_tools as pbt
import networkx as nx

from pybel.dsl import Abundance, BiologicalProcess, CentralDogma, ListAbundance, Reaction

from pathme.constants import REACTOME_BEL, KEGG_BEL, WIKIPATHWAYS_BEL, PATHME_DIR

In [3]:
pybel.get_version()

'0.13.1'

### Load Data Set 1: Input Scores

In [4]:
def munge_labels(label):
    """Process ene"""
    remove_set = ['*', ' ', '|', '-', '"', "'"]
    
    label = str(label).lower()
    
    for symb in remove_set:
        if symb in label:
            label = label.replace(symb, '')
    
    if '/' in label:
        label = tuple(set(label.split('/')))
        if len(label) == 1:
            label = label[0]
    
    return label


def parse_set1(path):
    
    wb = load_workbook(filename = path)

    sheet_titles = []
    omics_data = defaultdict(lambda:defaultdict(lambda:set()))
    omics_labels = defaultdict(lambda:set())

    for sheet in wb:
        cell_value = sheet['A3'].value

    #     if "Expression data (FC) of the differentially expressed" in sheet['A1'].value:
    #         sheet_title = sheet['A1'].value.split("Expression data (FC) of the differentially expressed ",1)[1]
    #         sheet_title = sheet_title.split(" of HepG2 cells after treatment with ")
    #         sheet_title[1] = sheet_title[1].replace(". Statistical significance (p value < 0.05) is indicated.", "").replace(" CsA for", "")
    #         sheet_titles.append(sheet_title)

        if cell_value and ("Significant " in cell_value or "Metabolite" == cell_value):
            if  cell_value == "Metabolite":
                sheet_title = ("Metabolite", '3 µM', ' 24h or 72h')
                min_row = 3

            else:
                sheet_title = cell_value.split("Significant ",1)[1]
                sheet_title = sheet_title.split(" CsA ")
                sheet_title.append(sheet_title[1].split(" ")[0] + ' h')
                sheet_title[1] = sheet_title[0].split(" ")[1]+ ' µM'
                sheet_title[0] = sheet_title[0].split(" ")[0]
                min_row = 4

            for col in sheet.iter_cols(min_row=min_row):
                col_label = col[0].value
                sheet_omic = sheet_title[0]

                if col_label in ['MicroRNA', 'hgnc symbol', 'Metabolite']:
                    omics_labels[sheet_omic.lower()].update(munge_labels(cell.value) for cell in col[1:])

            sheet_titles.append(sheet_title)

    return omics_labels
    
dataset1_omics_labels = parse_set1(os.path.join(dir_path, 'validation', 'set1.xlsx'))
dataset1_omics_labels

defaultdict(<function __main__.parse_set1.<locals>.<lambda>()>,
            {'genes': {'',
              'mier2',
              'gpcpd1',
              'jdp2',
              'sec63',
              'nt5c3',
              'gtf2e2',
              'pecr',
              'gna13',
              'carhsp1',
              'fam50a',
              'sox4',
              'hps3',
              'cno',
              'fkbp11',
              'hivep1',
              'rnf11',
              'elf4',
              'sox12',
              'rin3',
              'cenpn',
              'pqlc3',
              'dhcr7',
              'slc25a16',
              'tmem121',
              'nsrp1',
              'rnf7',
              'tmem219',
              'aldh5a1',
              'fdxr',
              'me2',
              'klhl18',
              'shank2',
              'cdk7',
              'matn3',
              'ccl16',
              'smcr7',
              'coq9',
              'gpr31',
              'cd36',
         

In [5]:
print(f'Total number of genes: ({len(dataset1_omics_labels["genes"])})')

print(f'Total number of metabolites: ({len(dataset1_omics_labels["metabolite"])})')

print(f'Total number of miRNAs: ({len(dataset1_omics_labels["micrornas"])})')

total_entities = len(dataset1_omics_labels["metabolite"]) + len(dataset1_omics_labels["micrornas"]) + len(dataset1_omics_labels["genes"])

print(total_entities)

Total number of genes: (4942)
Total number of metabolites: (21)
Total number of miRNAs: (100)
5063


## PathMe Imports

### Graph Universe

In [6]:
# Modify Python Typing arguments if you are using Python < 3.6
# PyBEL union method cannot be used because empty nodes are discarded
def get_nodes_in_database(folder):
    """Merge all the python pickles in a given folder and returns the corresponding BELGraph."""  
    database_networks = [
        pybel.from_pickle(os.path.join(folder, path))
        for path in os.listdir(folder)
        if path.endswith('.pickle')
    ]
    
    return {
        node
        for network in database_networks
        for node in network.nodes()
    }

kegg_nodes = get_nodes_in_database(KEGG_BEL)
reactome_nodes = get_nodes_in_database(REACTOME_BEL)
wikipathways_nodes = get_nodes_in_database(WIKIPATHWAYS_BEL)

#### Manual Curation
Since some of the entities were not categorized with their corresponding BEL function (e.g., abundance, gene) due to abnormalities in harmonization, we have manually curated them and assigned them to the right modality.

This data can be used by the original database to correct the node label in their corresponding pathway files.

In [7]:
# Entities in WikiPathways that required manual curation
WIKIPATHWAYS_BIOL_PROCESS = {'lipid biosynthesis', 'hsc survival', 'glycolysis & gluconeogenesis', 'triacylglyceride  synthesis', 'wnt canonical signaling', 'regulation of actin skeleton', 'fatty acid metabolism', 'mrna processing major splicing pathway', 'senescence', 'monocyte differentiation', 'pentose phosphate pathway', 'ethanolamine  phosphate', 'hsc differentiation', 'actin, stress fibers and adhesion', 'regulation of actin cytoskeleton', 's-phase progression', 'g1-s transition', 'toll-like receptor signaling pathway', 'regulation of  actin cytoskeleton', 'proteasome degradation', 'apoptosis', 'bmp pathway', 'ampk activation', 'g1/s checkpoint arrest', 'mapk signaling pathway', 'chromatin remodeling and  epigenetic modifications', 'wnt signaling pathway', 'ros production', 'erbb signaling pathway', 'shh pathway', 'inflammation', 'dna replication', 'mrna translation', 'oxidative stress', 'cell cycle checkpoint activation', 'gi/go pathway', 'wnt pathway', 'g1/s transition of mitotic cell cycle', 'modulation of estrogen receptor signalling', 'dna repair', 'bmp canonical signaling', 'igf and insuline signaling', 'unfolded protein response', 'cell death', 'p38/mapk  pathway', 'glycogen metabolism', 'gnrh signal pathway', 'the intra-s-phase checkpoint mediated arrest of cell cycle progression', 'tca cycle', 'mtor protein kinase signaling pathway', 'proteasome  degradation pathway', 'morphine metabolism', 'hsc aging', 'gastric pepsin release', 'parietal cell production', 'prostaglandin pathway', 'cell cycle (g1/s)  progression', 'notch pathway', 'g2/m progression', 'wnt signaling', 'cell adhesion', 'cell cycle progression', 'egfr pathway', 'cell cycle', 'angiogenesis', 'g2/m-phase checkpoint', 'hsc self renewal', '26s proteasome  degradation', 'mapk signaling', 'immune system up or down regulation', 'm-phase progression', 'insulin signaling', 'nf kappa b pathway', 'cell cycle  progression', 'gi pathway', 'cd45+ hematopoietic-    derived cell    proliferation', "kreb's cycle", 'glycogen synthesis', 'apoptosis pathway', 'g1/s progression', 'inflammasome activation', 'melanin biosynthesis', 'proteasomal degradation', 'g2/m checkpoint arrest', 'g1/s cell cycle transition', 'dna damage response', 'gastric histamine release'}
WIKIPATHWAYS_METAB = {'2,8-dihydroxyadenine', '8,11-dihydroxy-delta-9-thc', 'adp-ribosyl', 'cocaethylene', 'dhcer1p', 'ecgonidine', 'f2-isoprostane', 'fumonisins b1', 'iodine', 'l-glutamate', 'lactosylceramide', 'methylecgonidine', 'n-acetyl-l-aspartate', 'nad+', 'nadph oxidase', 'neuromelanin', 'nicotinic acid (na)', 'nmn', 'pip2', 'sphingomyelin', 'thf'}
WIKIPATHWAYS_NAME_NORMALIZATION = {"Ca 2+": "ca 2+", "acetyl coa": "acetyl-coa", "acetyl-coa(mit)": "acetyl-coa", "h20": "h2o"}

# Entities in Reactome that required manual curation
BLACK_LIST_REACTOME = {"5'"}
REACTOME_PROT = {'phospho-g2/m transition proteins', 'integrin alpha5beta1, integrin alphavbeta3, cd47', 'food proteins', 'activated fgfr2', 'adherens junction-associated proteins', 'pi3k mutants,activator:pi3k', 'prolyl 3-hydroxylases', 'gpi-anchored proteins', 'c3d, c3dg, ic3b', 'c4s/c6s chains', 'activated fgfr1 mutants and fusions', 'activated fgfr3 mutants', 'protein', 'cyclin a2:cdk2 phosphorylated g2/m transition protein', 'c4c, c3f', 'activated raf/ksr1', 'activated fgfr1 mutants', 'g2/m transition proteins', 'lman family receptors', 'cyclin', 'usp12:wdr48:wdr20,usp26', 'proteins with cleaved gpi-anchors', 'activated fgfr2 mutants', 'c4d, ic3b', 'c5b:c6:c7, c8, c9', 'cyclin a1:cdk2 phosphorylated g2/m transition protein', 'genetically or chemically inactive braf', 'il13-downregulated proteins', 'activated fgfr4 mutants', 'rna-binding protein in rnp (ribonucleoprotein) complexes', 'effector proteins', 'usp3, saga complex', 'dephosphorylated "receiver" raf/ksr1'}

#### Methods used in this notebook to process node information and create a set for each modality/database

In [8]:
def process_reactome_multiple_genes(genes):
    """Process a wrong ID with multiple identifiers"""
    gene_list = []
    for counter, gene in enumerate(genes):
        
        # Strip the ' gene' prefix
        gene = gene.strip().strip(' gene').strip(' genes')
        
        # First element is always OK
        if counter == 0:
            gene_list.append(gene)
        
        # If the identifier starts the same than the first one, it is right
        elif gene[:2] == genes[0][:2]:
            gene_list.append(gene)
        
        # If the identifier is longer than 2 it is a valid HGNC symbol
        elif len(gene) > 2:
            gene_list.append(gene)
 
        # If they start different, it might have only a number (e.g., 'ABC1, 2, 3') so it needs to be appended
        elif gene.isdigit():
            gene_list.append(genes[0][:-1] + gene)
        
        # If the have only one letter (e.g., HTR1A,B,D,E,F,HTR5A)
        elif len(gene) == 1:
            gene_list.append(genes[0][:-1] + gene)
            
    return gene_list
    
def munge_reactome_gene(gene):
    """Process Reactome gene"""
    if "," in gene:
        return process_reactome_multiple_genes(gene.split(","))
        
    elif "/" in gene:
        return process_reactome_multiple_genes(gene.split("/"))
    
    return gene

def calculate_database_sets(nodes, database):
    """Calculate node sets for each modality in the database"""
    gene_nodes = set()
    mirna_nodes = set()
    metabolite_nodes = set()
    bp_nodes = set()
    
    for node in nodes:
        
        if isinstance(node, ListAbundance) or isinstance(node, Reaction) or not node.name:
            continue
                
        # Lower case name and strip quotes or white spaces
        name = node.name.lower().strip('"').strip()
        
        # Dealing with Genes/miRNAs
        if isinstance(node, CentralDogma):
            
            ##################
            # miRNA entities #
            ##################
            
            if name.startswith("mir"):
                
                # Reactome preprocessing to flat multiple identifiers
                if database == 'reactome':
                    reactome_cell = munge_reactome_gene(name)
                    if isinstance(reactome_cell, list):
                        for name in reactome_cell:
                            mirna_nodes.add(name.replace("mir-", "mir"))
                    else:
                        mirna_nodes.add(name.strip(' genes').replace("mir-", "mir"))
                        
                    continue
                
                mirna_nodes.add(name.replace("mir-", "mir"))
                
            ##################
            # Genes entities #
            ##################
            
            else:
                # Reactome preprocessing to flat multiple identifiers
                if database == 'reactome':
                    reactome_cell = munge_reactome_gene(name)
                    if isinstance(reactome_cell, list):
                        for name in reactome_cell:
                            if name in BLACK_LIST_REACTOME: # Filter entities in black list
                                continue
                            elif name.startswith("("): # remove redundant parentheses
                                name = name.strip("(").strip(")")
                                
                            gene_nodes.add(name)
                    else:
                        gene_nodes.add(name)
                    continue
                    
                # WikiPathways and KEGG do not require any processing of genes
                if name in WIKIPATHWAYS_BIOL_PROCESS:
                    bp_nodes.add(name)
                    continue
                gene_nodes.add(name)
         
        #######################
        # Metabolite entities #
        #######################
        
        elif isinstance(node, Abundance):
            
            if database == 'wikipathways':
                # Biological processes that are captured as abundance in BEL since they were characterized wrong in WikiPathways
                if name in WIKIPATHWAYS_BIOL_PROCESS:
                    bp_nodes.add(name)
                    continue

                elif node.namespace in {'WIKIDATA', 'WIKIPATHWAYS', 'REACTOME'} and name not in WIKIPATHWAYS_METAB:
                    bp_nodes.add(name)
                    continue
                    
                # Fix naming in duplicate entity
                if name in WIKIPATHWAYS_NAME_NORMALIZATION:
                    name = WIKIPATHWAYS_NAME_NORMALIZATION[name]
                    
            elif database == 'reactome':
                # Curated proteins that were coded as metabolites
                if name in REACTOME_PROT:
                    gene_nodes.add(name)
                    continue
                
                # Flat multiple identifiers (this is not trivial because most of ChEBI names contain commas, 
                # so a clever way to fix some of the entities is to check that all identifiers contain letters)
                elif "," in name and all(
                    string.isalpha() 
                    for string in name.split(",")
                ):
                    for string in name.split(","):
                        metabolite_nodes.add(name)
                    continue
                    
            metabolite_nodes.add(name)

        #################################
        # Biological Processes entities #
        #################################
        
        elif isinstance(node, BiologicalProcess):
            if name.startswith('title:'):
                name = name[6:] # KEGG normalize
            
            bp_nodes.add(name)
        
    return gene_nodes, mirna_nodes, metabolite_nodes, bp_nodes

In [9]:
kegg_genes, kegg_mirna, kegg_metabolites, kegg_bps = calculate_database_sets(
    kegg_nodes, 'kegg'
)

reactome_genes, reactome_mirna, reactome_metabolites, reactome_bps = calculate_database_sets(
    reactome_nodes, 'reactome'
)
wikipathways_genes, wikipathways_mirna, wikipathways_metabolites, wikipathways_bps = calculate_database_sets(
    wikipathways_nodes, 'wikipathways'
)

####  Entity count for each modality in each database

In [10]:
print(f'Total number of genes: KEGG ({len(kegg_genes)}), Reactome ({len(reactome_genes)}), WikiPathways ({len(wikipathways_genes)})')

print(f'Total number of metabolites: KEGG ({len(kegg_metabolites)}), Reactome ({len(reactome_metabolites)}), WikiPathways ({len(wikipathways_metabolites)})')

print(f'Total number of miRNAs: KEGG ({len(kegg_mirna)}), Reactome ({len(reactome_mirna)}), WikiPathways ({len(wikipathways_mirna)})')

print(f'Total number of Biological Processes: KEGG ({len(kegg_bps)}), Reactome ({len(reactome_bps)}), WikiPathways ({len(wikipathways_bps)})')

Total number of genes: KEGG (7283), Reactome (6328), WikiPathways (3025)
Total number of metabolites: KEGG (4041), Reactome (2570), WikiPathways (454)
Total number of miRNAs: KEGG (149), Reactome (11), WikiPathways (68)
Total number of Biological Processes: KEGG (418), Reactome (2101), WikiPathways (102)


## Dataset label mapping to PathMe

In [11]:
def check_substrings(dataset_nodes, db_nodes):
    intersection_close = set()
    for entity in dataset_nodes:
        if isinstance(entity, tuple):
            for subentity in entity:
                for entity_db in db_nodes:
                    if entity_db in subentity or subentity in entity_db:
                        intersection_close.add(entity_db)
        else:
            for entity_db in db_nodes:
                if entity_db in entity or entity in entity_db:
                        intersection_close.add(entity_db)
    return intersection_close

### Dataset labels

In [12]:
kegg_all_omics_labels = set.union(kegg_mirna, kegg_metabolites, kegg_genes)
wikipathways_all_omics_labels = set.union(wikipathways_mirna, wikipathways_metabolites, wikipathways_genes)
reactome_all_omics_labels = set.union(reactome_mirna, reactome_metabolites, reactome_genes)
 
all_omics_labels_dataset1 = set()
for labels in dataset1_omics_labels.values():
    all_omics_labels_dataset1.update(set(labels))
                                         
all_omics_labels_dataset1

{'',
 'mier2',
 'gpcpd1',
 'jdp2',
 'sec63',
 'nt5c3',
 'parp11',
 'gtf2e2',
 'rai14',
 'arhgef3',
 'pecr',
 'unc119b',
 'gna13',
 'carhsp1',
 'ahr',
 'dag1',
 'bcl2l1',
 'gtf2h4',
 'rbks',
 'atp5d',
 'znf516',
 'dock4',
 'fam50a',
 'cdc6',
 'ccne1',
 'atf1',
 'sox4',
 'hps3',
 'cno',
 'fkbp11',
 'irf2bpl',
 ('mrpl12', 'slc25a10'),
 'c2orf44',
 'hivep1',
 'pm20d2',
 'pnpt1',
 'homer2',
 'rnf11',
 'elf4',
 'c3orf23',
 'sox12',
 'itch',
 'pcbp4',
 'fes',
 'notum',
 'rin3',
 'cenpn',
 'hcst',
 'pqlc3',
 'dhcr7',
 'slc25a16',
 ('hsamir4233p', 'rnomir423', 'mmumir4233p'),
 'cenpe',
 'tmem121',
 'nsrp1',
 'rnf7',
 'cdc7',
 'hes4',
 'pdxdc1',
 'tmem219',
 'aldh5a1',
 'fdxr',
 'ppfibp2',
 'me2',
 'znf621',
 'blcap',
 'klhl18',
 'shank2',
 'tspan8',
 'apof',
 'cdk7',
 'matn3',
 'ccl16',
 'ifit3',
 'patl1',
 'smcr7',
 'coq9',
 'ckap2l',
 'tsc22d1',
 'gpr31',
 'cd36',
 'ndufa9',
 '38961',
 'shb',
 'tbl3',
 'znf792',
 'gdi1',
 'wars2',
 ('rnomir299', 'hsamir2995p', 'mmumir299'),
 'cpt2',
 'znf248'

### KEGG

In [13]:
len(kegg_genes.intersection(dataset1_omics_labels['genes']))

1999

In [14]:
len(kegg_metabolites.intersection(dataset1_omics_labels['metabolite']))

8

In [15]:
print(len(kegg_mirna.intersection(dataset1_omics_labels['micrornas'])))
print(len(check_substrings(dataset1_omics_labels['micrornas'], kegg_mirna)))

0
20


In [16]:
kegg_mapping = kegg_all_omics_labels.intersection(all_omics_labels_dataset1)
len(kegg_mapping)

2007

In [17]:
print(len(kegg_mapping)/total_entities)

0.396405293304365


### Reactome

In [18]:
len(reactome_genes.intersection(dataset1_omics_labels['genes']))

1387

In [19]:
len(reactome_metabolites.intersection(dataset1_omics_labels['metabolite']))

2

In [20]:
print(len(reactome_mirna.intersection(dataset1_omics_labels['micrornas'])))
len(check_substrings(dataset1_omics_labels['micrornas'], reactome_mirna))

0


4

In [21]:
len(reactome_all_omics_labels.intersection(all_omics_labels_dataset1))

1393

### WikiPathways

In [22]:
len(wikipathways_genes.intersection(dataset1_omics_labels['genes']))

934

In [23]:
len(wikipathways_metabolites.intersection(dataset1_omics_labels['metabolite']))

11

In [24]:
print(len(wikipathways_mirna.intersection(dataset1_omics_labels['micrornas'])))
len(check_substrings(dataset1_omics_labels['micrornas'], wikipathways_mirna))

0


11

In [25]:
len(wikipathways_all_omics_labels.intersection(all_omics_labels_dataset1))

945

### All db mapping

In [26]:
all_omics_labels_db = set.union(kegg_all_omics_labels, reactome_all_omics_labels)

In [27]:
total_entities_db = len(all_omics_labels_db.intersection(all_omics_labels_dataset1))
print(total_entities_db)

print(total_entities_db/total_entities)

2479
0.48963065376259135


## Score Diffusion with diffuPy: Dataset as input + PathMe as background graph

In [28]:
from diffuPy.diffuse import diffuse

from diffuPy.matrix import Matrix, LaplacianMatrix

### Input  Matrix: Dataset

In [29]:
kegg_all_omics_labels = set.union(wikipathways_mirna, wikipathways_metabolites, wikipathways_genes)
wikipathways_all_omics_labels = set.union(wikipathways_mirna, wikipathways_metabolites, wikipathways_genes)
reactome_all_omics_labels = set.union(wikipathways_mirna, wikipathways_metabolites, wikipathways_genes)

all_omics_labels = set.union(kegg_all_omics_labels, wikipathways_all_omics_labels, reactome_all_omics_labels) 

input_mat = Matrix(rows_labels=all_omics_labels, cols_labels=['Dataset 1'], init=1)

In [30]:
print(len(input_mat.mat))

3542


### Kernel Matrix: PathMe Universe Graph

#### Graph Universe

In [31]:
def get_db_graph_universe(folder):
    """Merge all the python pickles in a given folder and returns the corresponding BELGraph."""  
    database_networks = [
        pybel.from_pickle(os.path.join(folder, path))
        for path in os.listdir(folder)
        if path.endswith('.pickle')
    ]
    
    return pybel.struct.union(database_networks)

def get_pathme_graph_universe():
    """Merge all the python pickles in a given folder and returns the corresponding BELGraph."""  
    if 'pathme_universe.pickle' in os.listdir(PATHME_DIR):
        return pybel.from_pickle(os.path.join(PATHME_DIR, 'pathme_universe.pickle'))

    else:
        kegg_graph = get_db_graph_universe(KEGG_BEL)
        reactome_graph = get_pathme_graph_universe(REACTOME_BEL)
        wikipathways_graph = get_pathme_graph_universe(WIKIPATHWAYS_BEL)
        
        pathme_universe = pybel.struct.union([kegg_graph, reactome_graph, wikipathways_graph])
        pathme_universe = pybel.struct.union(database_networks)
        pybel.to_pickle(pathme_universe, PATHME_DIR)
        return pathme_universe



In [32]:
pathme_graph_universe = get_pathme_graph_universe()

In [33]:
pathme_graph_universe.graph['pathway_id'] = 'pathme_graph_universe'

In [34]:
pathme_graph_universe.summarize()

Fc epsilon RI signaling pathway v1.0.0
Number of Nodes: 49602
Number of Edges: 97866
Network Density: 3.98E-05
Number of Components: 6570
Number of Warnings: 0


In [35]:
background_mat = LaplacianMatrix(pathme_graph_universe)

/Users/jmarinllao/Documents/CREB/diffuPy/src/diffuPy/miscellaneous.py:19: UserWarning: Graph must be undirected, so it is converted to undirected.
  warnings.warn('Graph must be undirected, so it is converted to undirected.')
Columns labels are assigned to rows since duplicate labels is true.


In [36]:
background_mat

In [37]:
input_mat = input_mat.match_missing_rows(background_mat.rows_labels, 0)

17815
[[1]
 [1]
 [1]
 ...
 [0]
 [0]
 [0]]


In [38]:
print(input_mat)


matrix  
  [[1]
 [1]
 [1]
 ...
 [0]
 [0]
 [0]] 
 row labels: 
  ['kpnb1', 'aflatoxin b1 8,9-dihydrodiol', 'mapkapk2', 'gna13', 'ahr', 'bcl2l1', 'dag1', 'prkaca', 'asmt', 'itgax', 'ccne1', 'atf1', 'col10a1', 'chrm5', 'slit2', 'crebbp', 'csk', 'cst5', 'iron(2+)', 'pnpt1', 'smarcc2', 'pnmt', 'ddx1', 'elf4', 'jak3', 'itch', 'il2', 'fes', 'sytl4', '3-oxohexanoyl-coa', 'dhcr7', "2'-deoxyadenosine", 'prkaa1', 'ntf3', 'ensg00000275042', 'nppa', 'pla2g4a', 'h2o', 'cair', 'nqo1', 'il1b', 'tnfrsf1b', 'tap1', 'cdk7', 'ifngr2', 'prkd1', 'stub1', 'drd5', 'ndufa10', 'atp5pd', 'cd36', 'ndufa9', 'tmed7', 'thioredoxin disulfide', 'shb', 'csnk1a1l', 'vegfa', 'gdi1', 'o-acetylated gd3', 'abi1', 'atp5f1d', 'ngf', 'kdr', 'grk6', 'lyn', 'phosphocholine', 'dvl1', 'nodal', 'ugt2a3', 'ump', 'krtap8-1', 'rab27a', 'cd28', 'pdgfb', 'cytochrome b', 'hla-b', 'dmac2l', 'rfc1', 'dgat1', 'glul', 'chst8', 'bdh1', 'socs5', 'camp', 'map3k1', 'sp1', '6-hitc', 'tnc', 'casp8', 'mccc1', 'cfp', 'cd48', 'hus1b', 'map2k4', 'bmf

In [42]:
print(len(input_mat.rows_labels))

print(len(background_mat.mat))

21357
21357
49602


### Compute diffusion scores

In [40]:
diffuse(input_mat, 'ml', K = background_mat)

KeyError: 'inpp5a'